In [1]:
import torch
import torch.nn as nn

import numpy as np

#### Написать на PyTorch forward и backward полносвязного слоя без использования autograd


In [2]:
def relu(x):
   return np.maximum(0,x)
   
def relu_dash(x, leak = 0):                    
    return np.where(x <= 0, leak, 1)

class NN(nn.Module):
    def __init__(self, input_dim: int, output_dim: int):
        super().__init__()
        self.linear = nn.Linear(in_features=input_dim, out_features=output_dim)
        self.activation = relu
        self.saved_tensors = None

    def forward(self, input_x):
        self.saved_tensors = input_x
        return self.activation(self.linear(input_x))

    def backward(self, grad_output):
        return grad_output * relu_dash(self.saved_tensors)

#### Написать 1-2 адаптивных оптимизатора

In [3]:
class Adagrad:
    def __init__(self, lr, model):
        self.accumulated = torch.zeros_like(model)
        self.lr = lr
        self.adapt_lr = lr
        self.model = model

    def step(self, grad):
        self.accumulated += grad**2
        self.adapt_lr = self.lr / torch.sqrt(self.accumulated)
        self.model -= self.adapt_lr * grad

In [4]:
class RMSprop:
    def __init__(self, rho, lr, model):
        self.accumulated = torch.zeros_like(model)
        self.rho = rho
        self.lr = lr
        self.adapt_lr = lr
        self.model = model

    def step(self, grad):
        self.accumulated += self.rho * self.accumulated + (1 - self.rho) * grad**2
        self.adapt_lr = self.lr / torch.sqrt(self.accumulated)
        self.model -= self.adapt_lr * grad

#### Решить задачу нахождения корней квадратного уравнения методом градиентного спуска

In [23]:
a, b, c = -3, 4, -1

f = lambda x: (a*x**2 + b*x + c)
g = lambda x: (2*a*x + b)

def solver(init_x):
    x = torch.tensor(init_x, dtype=torch.float32)
    grad = g(x)
    optim = Adagrad(0.001, x)
    for i in range(100):
        optim.step(grad)
        grad = g(optim.model)
    print(optim.model)

In [24]:
solver(2)

tensor(2.0186)
